# Variables

In [1]:
import sys
sys.path.append('..')
print(sys.path)

from dotenv import load_dotenv
from typing import TypedDict, Annotated
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage, AIMessage
from langchain.agents import create_tool_calling_agent, AgentExecutor
from pydantic import BaseModel, Field
from langchain.tools import Tool
from firecrawl import FirecrawlApp
import pandas as pd
import pprint
import os

from tools.fetching_description_from_huggingface import fetching_description_from_huggingface
load_dotenv(override = True)

MODEL = os.getenv("MODEL")
llm = ChatOpenAI(
    model=MODEL,
    temperature=0.2
)


['/usr/local/lib/python312.zip', '/usr/local/lib/python3.12', '/usr/local/lib/python3.12/lib-dynload', '', '/home/hieuclc/ise_chenla/venv/lib/python3.12/site-packages', '..']


# Testing

In [2]:
df_task = pd.read_csv("data/ISE - AutoCode Challenge 2 Public - Public task.csv")
df_model = pd.read_csv("data/ISE - AutoCode Challenge 2 Public - Model zoo.csv")

In [3]:
task = df_task["Task"][2]

In [4]:
def format_model_list(df):
    model_strings = []
    for i, row in df.iterrows():
        desc = str(row[1]).replace("{", "{{").replace("}", "}}")
        url = str(row[2])
        model_strings.append(f"{i + 1} - {desc} [More info]({url})")
    return "\n".join(model_strings)


## Tools

In [7]:
class AgentCodingState(TypedDict):
    problem_description: str
    model_description: str
    input_description: str
    output_description: str
    output_classes: str | None
    code: str | None

In [30]:
base_prompt = r"""
    You are a specialist in Machine Learning. Your task is to generate a **fully functional with all necessary imports and dependencies** codebase in **Python** that can be executed flawlessly.

    You will be provided with:
    - A problem description
    - An input specification
    - An output specification
    - A model description

    ### Input:
    - Problem description: {problem_description}
    - Model description: {model_description}
    - Input specification: {input_description}
    - Output specification: {output_description}
    - Output classes: {output_classes}

    ### Guidelines

    You **must** strictly follow the following guidelines:
    - The preprocessing step should be suitable for the data type.
    - The postprocessing step should notices the differences between the data returned by the model and the output requirements. You must extract and use the exact class labels as defined in the output specification.
    - Do **not invent new labels or translate** the class names. Use them exactly as given.

    Your implementation **must strictly follow** the structure below:
    1. **Imports**: All required libraries.
    2. **Preprocessing**: Handle and transform the input as defined.
    3. **Inference logic**: Use the described model for prediction. You **must** use `tqdm` or similar logging library to track progress.
    4. **Postprocessing**: Format or transform the raw output into the final result as described.

    You must **not** include any explanations, markdown, or logging outside what is required by the problem.

    Return **only** the complete Python codebase. Wrap it with:
    \`\`\`python
    # code here
    \`\`\`
    """

In [31]:
class OutputState(TypedDict):
    output_classes: str | None

In [32]:
def output_agent(state: OutputState) -> OutputState:
    prompt = r"""
        Your are a specialist in machine learning. Your task is to identify the absolute classes of the given problem description, following with an output description.
        ### Input:
        - Problem description: {problem_description}
        - Output description: {output_description}

        You must return an array, strictly following these guidelines:
        - Understand the context from the given problem description.
        - Extract the class names from the output description. You **must not** invent new labels or translate the class names. Use them exactly as given in the output description.
        - Create an array containing the classes.

        You must return only the array containing those classes, without any formatting.
    """

    prompt = prompt.format(
        problem_description = state["problem_description"],
        output_description = state["output_description"]
    )
    response = llm.invoke(prompt)
    return {**state, "output_classes": response.content}

In [33]:
output_state = {
    "problem_description": """Bối cảnh của vấn đề:
        Nhận dạng chữ viết tay là một bài toán cơ bản trong lĩnh vực học máy và xử lý ảnh, với nhiều ứng dụng thực tiễn như nhận dạng chữ số trên phiếu khảo sát, hóa đơn hay bài thi tự động. Việc phân loại chữ số viết tay thành số nguyên tố hoặc không giúp mở rộng khả năng ứng dụng trong các bài toán toán học tự động, kiểm tra bài tập, hoặc các ứng dụng giáo dục.

        Yêu cầu cụ thể cần đạt được:
        Xây dựng một hệ thống có khả năng nhận diện chữ số viết tay từ ảnh và xác định xem số đó có phải là số nguyên tố hay không.
        Đầu vào là ảnh chứa một chữ số viết tay.
        Đầu ra là nhãn phân loại nhị phân: """"nguyên tố"""" hoặc """"không nguyên tố"""".

        Định dạng dữ liệu đầu vào cho bài toán tổng thể:
        Một thư mục có tên là """"images"""" chứa các ảnh grayscale 28x28 pixel, mỗi ảnh chứa một chữ số viết tay từ 0 đến 9.

        Định dạng kết quả đầu ra mong muốn cho bài toán tổng thể:
        File output.csv mỗi hàng là kết quả dự đoán mỗi ảnh
        có các cột:
        file_name: tên file ảnh
        prediction: nhãn của ảnh:
        """"số nguyên tố"""" nếu số trong ảnh là số nguyên tố.
        """"không nguyên tố"""" nếu số trong ảnh không phải số nguyên tố.""",
    "output_description": """Định dạng kết quả đầu ra mong muốn cho bài toán tổng thể:
        File output.csv mỗi hàng là kết quả dự đoán mỗi ảnh
        có các cột:
        file_name: tên file ảnh
        prediction: nhãn của ảnh:
        """"số nguyên tố"""" nếu số trong ảnh là số nguyên tố.
        """"không nguyên tố"""" nếu số trong ảnh không phải số nguyên tố."""
}

In [34]:
new_state = output_agent(output_state)

In [35]:
new_state["output_classes"]

'["số nguyên tố", "không nguyên tố"]'

In [36]:
def coding_agent(state: AgentCodingState) -> AgentCodingState:
    prompt = base_prompt.format(
        problem_description = state["problem_description"],
        model_description = state["model_description"],
        input_description = state["input_description"],
        output_description = state["output_description"],
        output_classes = state["output_classes"]
    )
    response = llm.invoke(prompt)
    return {**state, "code": response.content}

In [37]:
prev_state = {
    "problem_description": """Bối cảnh của vấn đề:
        Nhận dạng chữ viết tay là một bài toán cơ bản trong lĩnh vực học máy và xử lý ảnh, với nhiều ứng dụng thực tiễn như nhận dạng chữ số trên phiếu khảo sát, hóa đơn hay bài thi tự động. Việc phân loại chữ số viết tay thành số nguyên tố hoặc không giúp mở rộng khả năng ứng dụng trong các bài toán toán học tự động, kiểm tra bài tập, hoặc các ứng dụng giáo dục.

        Yêu cầu cụ thể cần đạt được:
        Xây dựng một hệ thống có khả năng nhận diện chữ số viết tay từ ảnh và xác định xem số đó có phải là số nguyên tố hay không.
        Đầu vào là ảnh chứa một chữ số viết tay.
        Đầu ra là nhãn phân loại nhị phân: """"nguyên tố"""" hoặc """"không nguyên tố"""".

        Định dạng dữ liệu đầu vào cho bài toán tổng thể:
        Một thư mục có tên là """"images"""" chứa các ảnh grayscale 28x28 pixel, mỗi ảnh chứa một chữ số viết tay từ 0 đến 9.

        Định dạng kết quả đầu ra mong muốn cho bài toán tổng thể:
        File output.csv mỗi hàng là kết quả dự đoán mỗi ảnh
        có các cột:
        file_name: tên file ảnh
        prediction: nhãn của ảnh:
        """"số nguyên tố"""" nếu số trong ảnh là số nguyên tố.
        """"không nguyên tố"""" nếu số trong ảnh không phải số nguyên tố.""",
    "model_description": """Image Classification Model - Handwritten Digit Recognition (Color Images)
        📝 Mô tả
        Đây là một mô hình phân loại ảnh được huấn luyện để nhận diện các chữ số viết tay (từ 0 đến 9) trong ảnh màu. Mô hình sử dụng kiến trúc Vision Transformer (ViT) và được huấn luyện với tập dữ liệu gồm các chữ số viết tay được chụp hoặc scan dưới dạng ảnh màu.
        📌 Chi tiết các mô hình
        Phân loại ảnh chữ số viết tay từ 0 đến 9
        📥 Đầu vào
        Định dạng: Ảnh màu (RGB)
        Kiểu dữ liệu: PIL.Image.Image
        Kích thước ảnh: Mô hình sẽ tự resize theo cấu hình của ViTImageProcessor (thường là 224x224)
        Giá trị pixel: [0, 255] (chuẩn hóa tự động bởi processor)
        📤 Đầu ra
        Một số nguyên từ 0 đến 9, là kết quả phân loại tương ứng với chữ số xuất hiện trong ảnh.
        🛠 Yêu cầu thư viện
        Cài đặt các thư viện sau bằng pip:
        pip install transformers torch
        🧪 Sử dụng mô hình
        Dưới đây là đoạn mã mẫu để sử dụng mô hình:
        from transformers import ViTForImageClassification, ViTImageProcessor
        from PIL import Image
        import torch
        model_name = "thanhtlx/image_classification_01"
        model = ViTForImageClassification.from_pretrained(model_name)
        processor = ViTImageProcessor.from_pretrained(model_name)
        # Đọc ảnh cần phân loại
        image = Image.open("images.png")
        # Xử lý ảnh đầu vào
        inputs = processor(images=image, return_tensors="pt")
        # Dự đoán với mô hình
        with torch.no_grad():
            outputs = model(**inputs)
        # Lấy kết quả dự đoán
        predicted_class = outputs.logits.argmax(-1).item()
        print(f"prediction: {predicted_class}")""",
    "input_description": """Định dạng dữ liệu đầu vào cho bài toán tổng thể:
        Một thư mục có tên là """"images"""" chứa các ảnh grayscale 28x28 pixel, mỗi ảnh chứa một chữ số viết tay từ 0 đến 9.""",
    "output_description": """Định dạng kết quả đầu ra mong muốn cho bài toán tổng thể:
        File output.csv mỗi hàng là kết quả dự đoán mỗi ảnh
        có các cột:
        file_name: tên file ảnh
        prediction: nhãn của ảnh:
        """"số nguyên tố"""" nếu số trong ảnh là số nguyên tố.
        """"không nguyên tố"""" nếu số trong ảnh không phải số nguyên tố.""",
    "output_classes": new_state["output_classes"]
}

In [38]:
state = coding_agent(prev_state)

In [39]:
# state

In [40]:
print(state["code"])

```python
import os
import csv
from tqdm import tqdm
from PIL import Image
import torch
from transformers import ViTForImageClassification, ViTImageProcessor

# Load the pre-trained model and processor
model_name = "thanhtlx/image_classification_01"
model = ViTForImageClassification.from_pretrained(model_name)
processor = ViTImageProcessor.from_pretrained(model_name)
model.eval()

# Define the directory containing images
images_dir = "images"
# Define output CSV file
output_csv = "output.csv"

# Function to check if a number is prime
def is_prime(n):
    if n < 2:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False
    sqrt_n = int(n ** 0.5) + 1
    for i in range(3, sqrt_n, 2):
        if n % i == 0:
            return False
    return True

# Prepare list to hold results
results = []

# List all image files in the directory
image_files = [f for f in os.listdir(images_dir) if os.path.isfile(os.path.join(images_dir, f))]

# Process each image